# 각 층의 정보를 담은 geojson 파일 합병

In [1]:
! pip install geopandas

  Obtaining dependency information for geopandas from https://files.pythonhosted.org/packages/3e/cf/ede993ed7070c5487e6db550aad490178c06eaa48ad26ff5e5263f995fba/geopandas-0.14.1-py3-none-any.whl.metadata
  Obtaining dependency information for fiona>=1.8.21 from https://files.pythonhosted.org/packages/7f/27/b24c1610c7ae5716709321f04d38d7b8b71ed531f80df4f697b9ad99cfc3/fiona-1.9.5-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/51.1 kB ? eta -:--:--
     ---------------------------------------- 51.1/51.1 kB 2.6 MB/s eta 0:00:00
  Obtaining dependency information for pyproj>=3.3.0 from https://files.pythonhosted.org/packages/79/95/eb68113c5b5737c342bde1bab92705dabe69c16299c5a122616e50f1fbd6/pyproj-3.6.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for shapely>=1.8.0 from https://files.pythonhosted.org/packages/9e/39/029c441d8af32ab423b229c4525ce5ce6707318155b59634811a4c56f5c4/shapely-2.0.2-cp311-cp311-win_amd64.whl.metadata
   -

In [3]:
! pip install pandas

In [11]:
import geopandas as gpd
import os
import pandas as pd

# 현재 디렉토리에서 GeoJSON 파일 목록 가져오기
geojson_files = [file for file in os.listdir() if file.endswith('.geojson')]

# GeoDataFrame 초기화
gdfs = []  # Use a list to store GeoDataFrames

# 각 GeoJSON 파일을 GeoDataFrame에 추가
for file in geojson_files:
    filepath = os.path.join(os.getcwd(), file)
    # GeoJSON 파일을 읽어들일 때 GeometryDtype으로 변환
    gdf_temp = gpd.read_file(filepath, dtype={'geometry': 'geopandas.array.GeometryDtype'})
    
    # 데이터 유형이 GeometryDtype이 아니라면, GeometryDtype으로 변환
    if gdf_temp.geometry.dtype != 'geometry':
        gdf_temp['geometry'] = gdf_temp['geometry'].astype('geometry')
    
    # CRS 설정
    gdf_temp.crs = gdf_temp.crs if gdf_temp.crs else {'init': 'epsg:4326'}
    
    gdfs.append(gdf_temp)

# Concatenate the list of GeoDataFrames into one GeoDataFrame
gdf = pd.concat(gdfs, ignore_index=True)

# Convert 'geometry' column to standard geometry type
gdf['geometry'] = gdf['geometry'].astype('geometry')

# GeoDataFrame을 하나의 GeoJSON 파일로 저장
gdf.to_file("output_geojson.geojson", driver="GeoJSON")

In [34]:
! pip install fiona

In [41]:
! pip install geojson

  Obtaining dependency information for geojson from https://files.pythonhosted.org/packages/8e/1b/4f57660aa148d3e3043d048b7e1ab87dfeb85204d0fdb5b4e19c08202162/geojson-3.1.0-py3-none-any.whl.metadata


In [42]:
import fiona
from shapely.geometry import shape
import geojson
from geojson import Feature, Point

def geojson_to_osm(input_geojson, output_osm):
    # GeoJSON 파일 열기
    with fiona.open(input_geojson, 'r') as source:
        # OSM XML 헤더 작성
        osm_xml = '<?xml version="1.0" encoding="UTF-8"?>\n<osm version="0.6" generator="geojson_to_osm">\n'

        # 각 feature에 대해 OSM XML 노드 작성
        for feature in source:
            # 각 feature의 geometry와 properties 가져오기
            geom = shape(feature['geometry'])
            props = feature['properties']

            # 중심 좌표를 얻어와서 OSM 노드로 변환
            center = geom.centroid
            osm_xml += f'  <node id="{props["id"]}" lat="{center.y}" lon="{center.x}" version="1">\n'  # version 추가
            
            # Tags 작성
            for key, value in props.items():
                osm_xml += f'    <tag k="{key}" v="{value}"/>\n'
            
            osm_xml += '  </node>\n'

        # OSM XML 푸터 작성
        osm_xml += '</osm>'

        # OSM XML 파일 저장
        with open(output_osm, 'w', encoding='utf-8') as osm_file:
            osm_file.write(osm_xml)

# 입력 GeoJSON 파일 경로와 출력 OSM 파일 경로 지정
input_geojson = "output_geojson.geojson"
output_osm = "real_output.osm"

# GeoJSON을 OSM으로 변환
geojson_to_osm(input_geojson, output_osm)

print(f"Conversion completed. OSM file saved at: {output_osm}")

Conversion completed. OSM file saved at: real_output.osm
